


# Initialize generative AI: Gemini and pyttsx3 text-to-speech




In [ ]:
import google.generativeai as genai
import pyttsx3
import cv2
import numpy as np
import time
from PIL import Image, ImageGrab
import customtkinter as tk
import pyautogui
import secret

# Initialize Gemini Pro Vision
API_KEY = secret.API_KEY
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-pro-vision')

# Initialize pyttsx3
speaker = pyttsx3.init()
speaker.setProperty('rate', 120)
voices = speaker.getProperty('voices')
speaker.setProperty('voice', voices[1].id)

# Menu button function to start distraction hunting

In [ ]:
def activate():
    if user_input.get() == "":
        label3 = tk.CTkLabel(app, text = "You can't plan to do nothing!", font=my_font, fg_color='indigo') # Empty input not allowed
        canvas.create_window(400,425, window=label3)
        return
    keyword = user_input.get()
    canvas.delete("all") # Replace with new screen to signal to user distraction hunting has started
    button1 = tk.CTkButton(app, text="I'm done!", command = app.destroy, font=my_font)
    canvas.create_window(400,400, window=button1)
    label5 = tk.CTkLabel(app, text = f"Task: {keyword}", font=my_font, fg_color='indigo') # Displays inputted task
    canvas.create_window(400,300, window=label5)
    focus(keyword, streak) # actual distraction hunting function

# Distraction hunting loop (AI backend)

In [ ]:
# main distraction hunting manager
def focus(keyword, streak):
    screenshot() # take screenshot of user's screen
    picture = Image.open('s1.png')
    unrelated = judge(keyword, picture) # judge from the image whether the user is distarcted from their desired task
    if unrelated:
        streak += 1 # streak counter increases by 1 if screenshot implies that the user is distracted from their task
        if streak >= 2:
            taunt(keyword, picture) # streak of 2 or more shows that user is confirmed to be distracted for some time, so an audio reminder will be played
    else:
        streak = 0 # streak resets to 0 when user is deemed focused on their task again
    print(streak)
    app.after(1000, focus, keyword, streak) # constantly loops every 1 second while in distraction hunting phase

# record screen and take screenshots
def screenshot():
    ss = pyautogui.screenshot()
    ss.save("s1.png")

# send a text prompt based on the task keyword and make judgement based on the screenshot
def judge(keyword, image):
    qn = f'Answer YES or NO in caps, nothing else. Is this screenshot directly or indirectly related to "{keyword}"?' # first prompt
    print(qn)
    response = model.generate_content(contents=[qn, image])
    print(response.text)
    if response.text == "NO" or "NO" in response.text:
        return True
    qn = f'Answer YES or NO in caps, nothing else. Is the user doing something else unrelated to the task "{keyword}"?' # second prompt
    print(qn)
    response = model.generate_content(contents=[qn, image])
    print(response.text)
    if response.text == "YES" or "YES" in response.text:
        return True
    return False # user is deemed not distracted when both prompts infer that the user is doing the inputted task

# taunt the user with text-to-speech audio if it's clear they are distracted
def taunt(keyword, image):
    qn = f'This user is distracted right now. Reply below in ONE SENTENCE ONLY, NOTHING ELSE in a cheeky manner\
    to remind the user based on what they are currently doing and they are distracted from the task "{keyword}"'
    print(qn)
    response = model.generate_content(contents=[qn, image])
    print(response.text)
    speaker.say(response.text)
    speaker.runAndWait()

# Main app + menu interface

In [ ]:
app = tk.CTk()
app.title("Anti-Distraction Assistant")
my_font = tk.CTkFont(family="Courier", size=35, weight="bold")
user_input = tk.StringVar()
times = tk.StringVar()
keyword = ""
streak = 0
canvas = tk.CTkCanvas(app, bg='indigo', width=800, height=600)
canvas.pack()
label1 = tk.CTkLabel(app, text = "What do you plan to do today?", font=my_font, fg_color='indigo')
canvas.create_window(400,150, window=label1)
entry1 = tk.CTkEntry(app, textvariable = user_input, font=my_font, width=600)
canvas.create_window(400,300, window=entry1)
button = tk.CTkButton(app, text="Lock In", command = activate, font=my_font) # calls the activate() function
canvas.create_window(400,500, window=button)
app.mainloop()